# Corona Virus in the US

In [181]:
# data wrangle
import pandas as pd
import numpy as np

import json

### Aquire - deaths by county

In [384]:
df = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv")

In [385]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26199 entries, 0 to 26198
Data columns (total 6 columns):
date      26199 non-null object
county    26199 non-null object
state     26199 non-null object
fips      25803 non-null float64
cases     26199 non-null int64
deaths    26199 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 1.2+ MB


In [386]:
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


### Prepare

In [360]:
df.isnull().sum()

date        0
county      0
state       0
fips      396
cases       0
deaths      0
dtype: int64

In [361]:
df [df.fips.isnull()]

,date,county,state,fips,cases,deaths
416,2020-03-01,New York City,New York,NaN,1,0
418,2020-03-01,Unknown,Rhode Island,NaN,2,0
448,2020-03-02,New York City,New York,NaN,1,0
450,2020-03-02,Unknown,Rhode Island,NaN,2,0
482,2020-03-03,New York City,New York,NaN,2,0
...,...,...,...,...,...,...
25641,2020-04-01,Unknown,Rhode Island,NaN,179,8
25793,2020-04-01,Unknown,Tennessee,NaN,307,17
25952,2020-04-01,Unknown,Utah,NaN,19,3
25966,2020-04-01,Unknown,Vermont,NaN,23,9


In [362]:
df [df.fips.isnull()].county.value_counts()

Unknown          351
New York City     32
Kansas City       13
Name: county, dtype: int64

### Explore

In [363]:
states = df.groupby(df.state).sum()[["cases","deaths"]]

In [364]:
states["death_rate"] = round(states.deaths / states.cases * 100,2)

In [365]:
states.index.name = "state"

In [366]:
# states.reset_index(inplace=True)

In [367]:
states.sort_values("death_rate", ascending=False)

,cases,deaths,death_rate
state,,,
Northern Mariana Islands,14,1,7.14
Washington,50803,2628,5.17
Vermont,2321,108,4.65
Louisiana,35352,1418,4.01
Puerto Rico,1281,46,3.59
Oklahoma,3788,130,3.43
Georgia,26907,856,3.18
Oregon,5360,148,2.76
South Dakota,844,23,2.73


#### Add population data

State population data obtained from https://worldpopulationreview.com/states/

In [368]:
with open('data.json') as f:
    pop = json.load(f)

In [369]:
pop.keys()

dict_keys(['data'])

In [370]:
pop = pd.DataFrame(pop['data'])

In [371]:
df_state = pd.merge(states,pop, left_on="state", right_on="State")

In [372]:
df_state.head()

,cases,deaths,death_rate,rank,State,Pop,Growth,Pop2018,Pop2010,growthSince2010,Percent,density
0,7237,70,0.97,24,Alabama,4908621,0.0042,4887871,4785448,0.0257,0.0147,96.9221
1,994,10,1.01,49,Alaska,734002,-0.0047,737438,713906,0.0281,0.0022,1.2863
2,8278,106,1.28,14,Arizona,7378494,0.0288,7171646,6407774,0.1515,0.0221,64.9550
3,5106,65,1.27,33,Arkansas,3038999,0.0084,3013825,2921978,0.0400,0.0091,58.4030
4,65580,1325,2.02,1,California,39937489,0.0096,39557045,37320903,0.0701,0.1194,256.3727


In [373]:
df_state = df_state[["cases","deaths","death_rate","State","Pop"]]
df_state.columns = [x.lower() for x in df_state.columns]

In [374]:
df_state = df_state.set_index("state")

In [375]:
df_state.head()

,cases,deaths,death_rate,pop
state,,,,
Alabama,7237,70,0.97,4908621
Alaska,994,10,1.01,734002
Arizona,8278,106,1.28,7378494
Arkansas,5106,65,1.27,3038999
California,65580,1325,2.02,39937489


In [376]:
df_state.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, Alabama to Wyoming
Data columns (total 4 columns):
cases         52 non-null int64
deaths        52 non-null int64
death_rate    52 non-null float64
pop           52 non-null int64
dtypes: float64(1), int64(3)
memory usage: 2.0+ KB


In [377]:
df_state["cases_p_1000"] = round(df_state.cases / df_state["pop"] * 1000, 2)
df_state["deaths_p_1000"] = round(df_state.deaths / df_state["pop"] * 1000, 2)

In [378]:
df_state.sort_values("cases_p_1000", ascending=False).head()

,cases,deaths,death_rate,pop,cases_p_1000,deaths_p_1000
state,,,,,,
New York,547483,7709,1.41,19440469,28.16,0.40
New Jersey,114789,1576,1.37,8936574,12.84,0.18
Louisiana,35352,1418,4.01,4645184,7.61,0.31
Washington,50803,2628,5.17,7797095,6.52,0.34
Massachusetts,42264,463,1.10,6976597,6.06,0.07


In [379]:
df_state.sort_values("deaths_p_1000", ascending=False).head()

,cases,deaths,death_rate,pop,cases_p_1000,deaths_p_1000
state,,,,,,
New York,547483,7709,1.41,19440469,28.16,0.40
Washington,50803,2628,5.17,7797095,6.52,0.34
Louisiana,35352,1418,4.01,4645184,7.61,0.31
New Jersey,114789,1576,1.37,8936574,12.84,0.18
Vermont,2321,108,4.65,628061,3.70,0.17


> The `deaths_p_1000` is probably the number that is most reflective of the damage the virus is causing within a state. As all states have insufficient testing and it is truly unknown how much the virus has spread.  

### Texas Counties

In [389]:
df_texas = df [df.state == "Texas"]

In [393]:
df_texas.head()

,date,county,state,fips,cases,deaths
143,2020-02-12,Bexar,Texas,48029.0,1,0
154,2020-02-13,Bexar,Texas,48029.0,2,0
165,2020-02-14,Bexar,Texas,48029.0,2,0
176,2020-02-15,Bexar,Texas,48029.0,2,0
187,2020-02-16,Bexar,Texas,48029.0,2,0


In [392]:
df_texas.county.value_counts()

Bexar         50
Fort Bend     29
Harris        28
Collin        24
Montgomery    23
              ..
Live Oak       1
Camp           1
Clay           1
Anderson       1
Gonzales       1
Name: county, Length: 141, dtype: int64

In [400]:
df_texas [df_texas.county == "Bexar"]

,date,county,state,fips,cases,deaths
143,2020-02-12,Bexar,Texas,48029.0,1,0
154,2020-02-13,Bexar,Texas,48029.0,2,0
165,2020-02-14,Bexar,Texas,48029.0,2,0
176,2020-02-15,Bexar,Texas,48029.0,2,0
187,2020-02-16,Bexar,Texas,48029.0,2,0
199,2020-02-17,Bexar,Texas,48029.0,2,0
211,2020-02-18,Bexar,Texas,48029.0,2,0
223,2020-02-19,Bexar,Texas,48029.0,2,0
236,2020-02-20,Bexar,Texas,48029.0,2,0
250,2020-02-21,Bexar,Texas,48029.0,4,0
